# Folding and Unfolding Example.

In [1]:
import paragami

import autograd
from autograd import numpy as np
from autograd import scipy as sp

# Use the original scipy for functions we don't need to differentiate.
import scipy as osp

We will begin by considering a simple symmetric positive semi-definite matrix.  In general, we can write:

$$
A = \left[
\begin{matrix}
a_{11} & a_{12} & a_{13}  \\
a_{21} & a_{22} & a_{23}  \\
a_{31} & a_{32} & a_{33}  \\
\end{matrix}
\right]
$$

...although, of course, symmetry and positive semi-definiteness impose constraints on the entries $a_{ij}$ of $A$ which we will discuss below.

## Flattening and folding.

Let us first consider how to represent $A$ as a vector, and then as an unconstrained vector.

Because $A$ is symmetric, $a_{21} = a_{12}$, $a_{31} = a_{13}$, and $a_{23} = a_{32}$.  This means that to reproduce $A$ we only need to store the bold values:

$$
A = \left[
\begin{matrix}
\mathbf{a_{11}} & \mathbf{a_{12}} & \mathbf{a_{13}}  \\
a_{21} & \mathbf{a_{22}} & \mathbf{a_{23}}  \\
a_{31} & a_{32} & \mathbf{a_{33}}  \\
\end{matrix}
\right]
$$

If we wanted to represent $A$ as an flat array, we could write

$$
A_{flat} = 
\left[
\begin{matrix}
a_{flat,1} \\
a_{flat,2} \\
a_{flat,3} \\
a_{flat,4} \\
a_{flat,5} \\
a_{flat,6} \\
\end{matrix}
\right] =
\left[
\begin{matrix}
\mathbf{a_{11}} \\
\mathbf{a_{12}} \\
\mathbf{a_{22}} \\
\mathbf{a_{13}} \\
\mathbf{a_{23}} \\
\mathbf{a_{33}} \\
\end{matrix}
\right]
$$

and fully recover $A$ from $A_{flat}$.

Converting to and from $A$ and $A_{flat}$ can be done with the `flatten` method of a `paragami.PSDSymmetricMatrixPattern` pattern.  

For the moment, because we are simply re-stacking the values of $A$ into a vector, not transforming into an unconstrained space, we use the option `free=False`.  We will discuss the `free=True` option shortly.

In [2]:
# A sample positive semi-definite matrix.
a = np.eye(3) + np.random.random((3, 3))
a = 0.5 * (a + a.T)

# Define a pattern and fold.
a_pattern = paragami.PSDSymmetricMatrixPattern(size=3)
a_flat = a_pattern.flatten(a, free=False)

print('Now, a_flat contains the elements of a exactly as shown in the formula above.\n')
print('a:\n{}\n'.format(a))
print('a_flat:\n{}\n'.format(a_flat))

Now, a_flat contains the elements of a exactly as shown in the formula above.

a:
[[1.43029514 0.91026842 0.28169565]
 [0.91026842 1.96292882 0.67954451]
 [0.28169565 0.67954451 1.39334116]]

a_flat:
[1.43029514 0.91026842 1.96292882 0.28169565 0.67954451 1.39334116]



We can also convert from $A_{flat}$ back to $A$ by 'folding'.

In [3]:
print('Folding the flattened value recovers the original matrix.\n')
a_fold = a_pattern.fold(a_flat, free=False)
print('a:\n{}\n'.format(a))
print('a_fold:\n{}\n'.format(a_flat))

Folding the flattened value recovers the original matrix.

a:
[[1.43029514 0.91026842 0.28169565]
 [0.91026842 1.96292882 0.67954451]
 [0.28169565 0.67954451 1.39334116]]

a_fold:
[1.43029514 0.91026842 1.96292882 0.28169565 0.67954451 1.39334116]



## Free flattening and folding.

Not every length-six vector represents a valid positive semi-definite matrix.  If the attribute `validate` of a `paragami.PSDSymmetricMatrixPattern` is true, then folding automatically checks for validity.

In [4]:
print('The diagonal of a positive semi-definite matrix must not be less',
      'than 0, and folding checks this when validate=True, which it is by default:\n')
a_flat_bad = np.array([-1, 0, 0, 0, 0, 0])
print('A bad folded value: {}'.format(a_flat_bad))
try:
    a_fold_bad = a_pattern.fold(a_flat_bad, free=False)
except ValueError as err:
    print('Folding with a_pattern raised the following ValueError:\n{}'.format(err))

print('\nIf validate is false, folding will produce an invalid matrix without an error:\n')
a_pattern.validate = False
a_fold_bad = a_pattern.fold(a_flat_bad, free=False)
print('Folding a non-pd matrix with validate=False:\n{}'.format(a_fold_bad))

print('\nHowever, it will not produce a matrix of the wrong shape even when validate is False:\n')
a_flat_very_bad = np.array([1, 0, 0])
print('A very bad folded value: {}.'.format(a_flat_very_bad))
try:
    a_fold_very_bad = a_pattern.fold(a_flat_very_bad, free=False)
except ValueError as err:
    print('Folding with a_pattern raised the following ValueError:\n{}'.format(err))

# Let's set validate back to true.
a_pattern.validate = True

The diagonal of a positive semi-definite matrix must not be less than 0, and folding checks this when validate=True, which it is by default:

A bad folded value: [-1  0  0  0  0  0]
Folding with a_pattern raised the following ValueError:
Diagonal is less than the lower bound 0.0.

If validate is false, folding will produce an invalid matrix without an error:

Folding a non-pd matrix with validate=False:
[[-1.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]

However, it will not produce a matrix of the wrong shape even when validate is False:

A very bad folded value: [1 0 0].
Folding with a_pattern raised the following ValueError:
Wrong length for PDMatrix flat value.


Sometimes (e.g. when performing optimization) it is convenient to have a flattened value that can legally take any value of the correct length.  For a positive semi-definite matrix, this can be achieved by a Cholesky decomposition.  The details aren't important here, as `paragami` takes care of the transformation behind the scenes with the option `free=True`. 

We denote the flattened $A$ in the free parameterization as $A_{freeflat}$.  Of course, in general, $A_{freeflat} \ne A_{flat}$, though each is related to $A$ -- and, hence, to each other -- by a diffemorphism.

In [5]:
print('The free flat value a_freeflat is not immediately recognizable as a.\n')
a_freeflat = a_pattern.flatten(a, free=True)
print('a:\n{}\n'.format(a))
print('a_freeflat:\n{}\n'.format(a_freeflat))

print('However, it transforms correctly back to a when folded.\n')
a_freefold = a_pattern.fold(a_freeflat, free=True)
print('a_fold:\n{}\n'.format(a_freefold))

print('Any length-six vector will free fold back to a valid positive',
      'semi-definite matrix up to floating point error.',
      'Let\'s draw 100 random vectors, fold them, and check that this is true.\n')
# Draw random free vectors and confirm that they are positive semi definite.
def assert_is_pd(mat):
    eigvals = np.linalg.eigvals(mat)
    assert np.min(eigvals) >= -1e-8
for raw in range(100):
    a_rand_freeflat = np.random.normal(scale=2, size=(6, ))
    a_rand_fold = a_pattern.fold(a_rand_freeflat, free=True)
    assert_is_pd(a_rand_fold)


print('Note that you will get an incorrect values or errors if you mix',
      'free and non-free folding and flattening!',
      '\nIn general, validate cannot be relied upon to catch such errors.\n')
a_pattern.validate = False
a_fold_free_unfree_mix = a_pattern.fold(a_freeflat, free=False)
print('a_fold_free_unfree_mix:\n{}\n'.format(a_fold_bad))



The free flat value a_freeflat is not immediately recognizable as a.

a:
[[1.43029514 0.91026842 0.28169565]
 [0.91026842 1.96292882 0.67954451]
 [0.28169565 0.67954451 1.39334116]]

a_freeflat:
[0.17894041 0.76112615 0.16235011 0.23554143 0.42529939 0.07290737]

However, it transforms correctly back to a when folded.

a_fold:
[[1.43029514 0.91026842 0.28169565]
 [0.91026842 1.96292882 0.67954451]
 [0.28169565 0.67954451 1.39334116]]

Any length-six vector will free fold back to a valid positive semi-definite matrix up to floating point error. Let's draw 100 random vectors, fold them, and check that this is true.

Note that you will get an incorrect values or errors if you mix free and non-free folding and flattening! 
In general, validate cannot be relied upon to catch such errors.

a_fold_free_unfree_mix:
[[-1.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]



## Using flattening and folding for optimization.

Suppose we are interested in optimizing some function of $A$, say, a normal model in which the data $x_n \sim \mathcal{N}(0, A)$.  Specifically, Let the data be $X = \left(x_1, ..., x_N\right)$, where $x_n \in \mathbb{R}^3$, and write a loss function as

$$
\ell\left(X, A\right) =
    -\sum_{n=1}^N \log P(x_n | A) =
    \frac{1}{2}\sum_{n=1}^N \left(x_n^T A^{-1} x_n - \log|A|\right) 
$$

Let's simulate some data under this model.

In [34]:
np.random.seed(42)

num_obs = 1000

# True value of A
true_a = np.eye(3) * np.diag(np.array([1, 2, 3])) + np.random.random((3, 3)) * 0.1
true_a = 0.5 * (true_a + true_a.T)

# Data
def draw_data(num_obs, true_a):
    return np.random.multivariate_normal(
        mean=np.zeros(3), cov=true_a, size=(num_obs, ))

x = draw_data(num_obs, true_a)
print('X shape: {}'.format(x.shape))

def get_loss(x, a):
    num_obs = x.shape[0]
    a_inv = np.linalg.inv(a)
    a_det_sign, a_log_det = np.linalg.slogdet(a)
    assert a_det_sign > 0
    return 0.5 * (np.einsum('ni,ij,nj', x, a_inv, x) + num_obs * a_log_det)

print('Loss at true parameter: {}'.format(get_loss(x, true_a)))

X shape: (1000, 3)
Loss at true parameter: 2392.751922600241


We would like to minimize the function `loss` using tools like `scipy.optimize.minimize`.  Standard optimization functions take vectors, not matrices, as input, and often require the vector to take valid values in the entire domain.

As-written, our loss function takes a positive definite matrix as an input.  We can wrap the loss as a funciton of the free flattened value using the `paragami.FlattenedFunction` class.  That is, we want to define a function $\ell_{freeflat}$ so that

$$
\ell_{freeflat}(X, A_{freeflat}) = \ell(X, A).
$$


The resulting function can be passed directly to `autograd` and `scipy.optimize`, and we can estimate

$$
\hat{A}_{freeflat} := \mathrm{argmin}_{A_{freeflat}} \ell_{freeflat}(X, A_{freeflat})
$$

In [70]:
# The arguments mean we're flatting the function get_loss, using
# the pattern a_pattern, with free parameterization, and the paramater
# is the second one (argnums uses 0-indexing like autograd).
get_freeflat_loss = paragami.FlattenedFunction(
    original_fun=get_loss, patterns=a_pattern, free=True, argnums=1)

print('The two losses are the same when evalated on the folded and flat values:\n')
print('Original loss:\t\t{}'.format(get_loss(x, true_a)))
true_a_freeflat = a_pattern.flatten(true_a, free=True)
print('Free-flattened loss: \t{}'.format(
    get_freeflat_loss(x, true_a_freeflat)))

print('\nNow, use the flattened function to optimize with autograd.\n')

get_freeflat_loss_grad = autograd.grad(get_freeflat_loss, argnum=1)
get_freeflat_loss_hessian = autograd.hessian(get_freeflat_loss, argnum=1)

def get_optimum(x):
    loss_opt = osp.optimize.minimize(
        method='trust-ncg',
        x0=np.zeros(a_pattern.flat_length(free=True)),
        fun=lambda par: get_freeflat_loss(x, par),
        jac=lambda par: get_freeflat_loss_grad(x, par),
        hess=lambda par: get_freeflat_loss_hessian(x, par),
        options={'gtol': 1e-8, 'disp': True})
    return loss_opt

loss_opt = get_optimum(x)
print('Optimization successful: {}\nOptimal value: {}'.format(
    loss_opt.success, loss_opt.fun))

The two losses are the same when evalated on the folded and flat values:

Original loss:		2392.751922600241
Free-flattened loss: 	2392.751922600241

Now, use the flattened function to optimize with autograd.

         Current function value: 2390.646332
         Iterations: 9
         Function evaluations: 11
         Gradient evaluations: 10
         Hessian evaluations: 10
Optimization successful: False
Optimal value: 2390.6463320487683


The optimization was in the free flattened space, so to get the optimal value of $A$ we must fold it.   We can see that the optimal value is close to the true value of $A$, though it differs due to randomness in $X$.

In [69]:
optimal_freeflat_a = loss_opt.x
optimal_a = a_pattern.fold(optimal_freeflat_a, free=True)
print('True a:\n{}\n\nOptimal a:\n{}'.format(true_a, optimal_a))

True a:
[[1.03745401 0.07746864 0.03950388]
 [0.07746864 2.01560186 0.05110853]
 [0.03950388 0.05110853 3.0601115 ]]

Optimal a:
[[ 1.0688328   0.07771763  0.04877013]
 [ 0.07771763  1.89393526 -0.03098498]
 [ 0.04877013 -0.03098498  2.94476044]]


## Using flattening and folding for uncertainty.

Suppose we wanted to use the Hessian of the objective (the observed Fisher information) to estimate a frequentist confidence region for $A$.  Typically a covariance is of a vector, so we can write what we want in terms of $A_{flat}$:

$$
\mathrm{Cov}(A_{flat}) \approx \left(\frac{\partial^2 \ell_{flat}}{\partial A_{flat} \partial A_{flat}^T}\right)^{-1}.
$$

(Here, we define $\ell_{flat}$ analogously to how we defined $\ell_{freeflat}$ above.)  The covariance between two elements of $A_{flat}$ corresponds to that between two elements of $A$.  For example, using the notation given above,

$$
\mathrm{Cov}(a_{flat,1}, a_{flat,2}) = \mathrm{Cov}(a_{11}, a_{12}) = \mathrm{Cov}(a_{11}, a_{21})\\
\mathrm{Var}(a_{flat,4}) = \mathrm{Var}(a_{13}) = \mathrm{Var}(a_{31}),
$$
etc.

Let us see how we can use flattening and folding to make estimating these covariances more convenient.  

First, we note that we cannot use the Hessian of the objective with respect to the free flattened value of $A$.  That would give us a confidence region in the unconstrained paremeterization, whereas we want a confidence region for $A$ itself.

$$
\mathrm{Cov}(A_{freeflat}) \ne \mathrm{Cov}(A_{free})\\
\frac{\partial^2 \ell_{freeflat}}{\partial A_{freeflat} \partial A_{freeflat}^T} \ne
\frac{\partial^2 \ell_{flat}}{\partial A_{free} \partial A_{free}^T}.
$$

However, because $A_{flat}$ and $A_{freeflat}$ are related by a diffeomorphism (the unconstraining parameterzation), an optimal value of $\ell$ for $A_{freeflat}$ is also an optimal value of $A_{freeflat}$:

$$
\left. \frac{\partial \ell_{free}}{\partial A_{free}}\right|_{\hat{A}_{free}} =
\left(\frac{d A_{free}}{dA_{freeflat}}\right)^T
\left. \frac{\partial \ell_{freeflat}}{\partial A_{freeflat}}\right|_{\hat{A}_{freeflat}} = 0 \Rightarrow\\
\left. \frac{\partial \ell_{free}}{\partial A_{free}}\right|_{\hat{A}_{free}} = 0
$$

(This relationship won't hold in general between free and non-free flattened values, e.g. for simplexes, but it holds in this case.)  Consequently, we can simply calculate the Hessian at the $\hat{A}_{free}$ corresponding to the $\hat{A}_{freeflat}$ we already found.

In [68]:
# Get the loss as a function of the flattened parameter.
get_flat_loss = paragami.FlattenedFunction(
    original_fun=get_loss, patterns=a_pattern, free=False, argnums=1)
get_flat_loss_grad = autograd.grad(get_flat_loss, argnum=1)
get_flat_loss_hessian = autograd.hessian(get_flat_loss, argnum=1)

# Make sure that the optimal value for the free flattened parameter is
# also optimal for the flattened parameter.  In general this may not
# be true if the unconstraining parameterization changes the dimensionality,
# (as with a simplex for example).
optimal_flat_a = a_pattern.flatten(optimal_a, free=False)
print('Sanity check: the gradient is also zero with respect',
      'to the non-flattened parameter at the optimum.')
print('Norm of flattened gradient: {}'.format(
      np.linalg.norm(get_flat_loss_grad(x, optimal_flat_a))))

# Estimate the covariance of the flattened value using the Hessian at the optimum.
a_flattened_cov = np.linalg.inv(get_flat_loss_hessian(x, optimal_flat_a))

Sanity check: the gradient is also zero with respect to the non-flattened parameter at the optimum.
Norm of flattened gradient: 4.564048764418554e-08


The resulting covariance matrix is 6x6, which is the shape of the flattened vector.

In [65]:
print('Shape of the covariance matrix: {}'.format(a_flattened_cov.shape))
print('Length of the flattened vector: {}'.format(a_pattern.flat_length(free=False)))

Shape of the covariance matrix: (6, 6)
Length of the flattened vector: 6


This shape is inconvenient because it's not obvious visually which entry of the flattened vector corresponds to which element of $A$.  Again, we can use folding to put the estimated marginal standard deviations in a readable shape.

Because the result is not a valid covariance matrix, and we are just using the pattern for its shape, we set `verify` to `False`.

In [67]:
a_pattern.verify = False
a_sd = a_pattern.fold(np.sqrt(-np.diag(a_flattened_cov)), free=False)
print('The marginal standard deviations of the elements of A:\n{}'.format(a_sd))

The marginal standard deviations of the elements of A:
[[0.04779966 0.0450593  0.05612339]
 [0.0450593  0.08469936 0.07468698]
 [0.05612339 0.07468698 0.13169369]]


As a sanity check, we can compare this estimated covariance with the variability incurred by drawing new datasets and re-optimizing.

In [74]:
num_sims = 100
optimal_a_draws = np.empty((num_sims, ) + true_a.shape)
for sim in range(num_sims):
    new_x = draw_data(num_obs, true_a)
    new_loss_opt = get_optimum(new_x)
    optimal_a_draws[sim] = a_pattern.fold(new_loss_opt.x, free=True)


         Current function value: 2422.524083
         Iterations: 9
         Function evaluations: 11
         Gradient evaluations: 10
         Hessian evaluations: 10
         Current function value: 2401.509071
         Iterations: 9
         Function evaluations: 11
         Gradient evaluations: 10
         Hessian evaluations: 10
         Current function value: 2371.843023
         Iterations: 23
         Function evaluations: 25
         Gradient evaluations: 11
         Hessian evaluations: 11
         Current function value: 2407.932388
         Iterations: 8
         Function evaluations: 10
         Gradient evaluations: 9
         Hessian evaluations: 9
         Current function value: 2457.291141
         Iterations: 9
         Function evaluations: 11
         Gradient evaluations: 10
         Hessian evaluations: 10
         Current function value: 2359.277717
         Iterations: 8
         Function evaluations: 10
         Gradient evaluations: 9
         Hessian eval

         Current function value: 2447.284310
         Iterations: 22
         Function evaluations: 24
         Gradient evaluations: 9
         Hessian evaluations: 9
         Current function value: 2482.848468
         Iterations: 9
         Function evaluations: 11
         Gradient evaluations: 10
         Hessian evaluations: 10
         Current function value: 2463.578084
         Iterations: 9
         Function evaluations: 11
         Gradient evaluations: 10
         Hessian evaluations: 10
         Current function value: 2467.801090
         Iterations: 22
         Function evaluations: 24
         Gradient evaluations: 11
         Hessian evaluations: 11
         Current function value: 2386.433896
         Iterations: 9
         Function evaluations: 11
         Gradient evaluations: 10
         Hessian evaluations: 10
         Current function value: 2423.166918
         Iterations: 8
         Function evaluations: 10
         Gradient evaluations: 9
         Hessian eva

         Current function value: 2386.672065
         Iterations: 9
         Function evaluations: 11
         Gradient evaluations: 10
         Hessian evaluations: 10
         Current function value: 2444.762862
         Iterations: 9
         Function evaluations: 11
         Gradient evaluations: 10
         Hessian evaluations: 10
         Current function value: 2414.926022
         Iterations: 9
         Function evaluations: 11
         Gradient evaluations: 10
         Hessian evaluations: 10
         Current function value: 2392.389811
         Iterations: 9
         Function evaluations: 11
         Gradient evaluations: 10
         Hessian evaluations: 10
         Current function value: 2421.740006
         Iterations: 10
         Function evaluations: 12
         Gradient evaluations: 11
         Hessian evaluations: 11
         Current function value: 2355.855876
         Iterations: 23
         Function evaluations: 25
         Gradient evaluations: 11
         Hessian 

In [75]:
print('Actual standard deviation:\n{}'.format(np.std(optimal_a_draws, axis=0)))
print('Estimated standard deviation:\n{}'.format(a_sd))

Actual standard deviation:
[[0.04086037 0.05113332 0.05668691]
 [0.05113332 0.08274279 0.07181718]
 [0.05668691 0.07181718 0.12629836]]
Estimated standard deviation:
[[0.04779966 0.0450593  0.05612339]
 [0.0450593  0.08469936 0.07468698]
 [0.05612339 0.07468698 0.13169369]]
